In [3]:
from google.oauth2 import service_account
from google.cloud import vision
import io
import re
import numpy as np
import os
import xml.etree.ElementTree as ET 
from joblib import Parallel, delayed
import multiprocessing

In [4]:
def extract_clean_str(original_str):
    return re.sub('[^a-zA-Z0-9]+', ' ', original_str, flags=re.UNICODE).lower().strip().replace(' ', '')

def get_mapped(path):
    key_map, dict_idx = {}, 0
    for r, _, f in os.walk(dir_path):
        for file in f:
            if '.JPG' in file or '.jpg' in file:
                key_map[dict_idx] = file
                dict_idx += 1
    idxs = list(key_map.keys())          
    shuffled_idxs = np.random.randint(0, len(idxs), len(idxs))
    return key_map, shuffled_idxs

def fetch_from_xml(path, file):
    tree = ET.parse(os.path.join(path, file)).getroot()  
    return extract_clean_str(''.join([e.attrib['char'] for e in tree.iter(tag='character')]))

In [5]:
dir_path = 'val2017/'
dict_files, files_idx = get_mapped(dir_path)
shuffled_idx = np.random.randint(0, len(files_idx), len(files_idx))
file = '000000523957.jpg' #dict_files.get(shuffled_idx[65]) 
with io.open(os.path.join(dir_path, file), 'rb') as image_file:
    content = (image_file.read())

In [20]:
vision_client = vision.ImageAnnotatorClient()
image = vision.types.Image(content=content)
text_detection_response = vision_client.label_detection(image=image)
detected_labels = [label.description for label in text_detection_response.label_annotations]

In [21]:
detected_labels

['Outdoor recreation',
 'Recreation',
 'Longboard',
 'Skateboard',
 'Skateboarding Equipment',
 'Longboarding',
 'Sports',
 'Individual sports',
 'Skateboarding',
 'Boardsport']

In [17]:

text_detection_response.label_annotations

[mid: "/m/05b0n7k"
description: "Outdoor recreation"
score: 0.890464723110199
topicality: 0.890464723110199
, mid: "/m/06bm2"
description: "Recreation"
score: 0.8842983841896057
topicality: 0.8842983841896057
, mid: "/m/02nnq5"
description: "Longboard"
score: 0.8223562836647034
topicality: 0.8223562836647034
, mid: "/m/06_fw"
description: "Skateboard"
score: 0.8150969743728638
topicality: 0.8150969743728638
, mid: "/m/0bpn3c2"
description: "Skateboarding Equipment"
score: 0.791763961315155
topicality: 0.791763961315155
, mid: "/m/03bx07d"
description: "Longboarding"
score: 0.7709515690803528
topicality: 0.7709515690803528
, mid: "/m/06ntj"
description: "Sports"
score: 0.7704066038131714
topicality: 0.7704066038131714
, mid: "/m/08t0vy"
description: "Individual sports"
score: 0.7562195658683777
topicality: 0.7562195658683777
, mid: "/m/06zfw"
description: "Skateboarding"
score: 0.7472692728042603
topicality: 0.7472692728042603
, mid: "/m/09qkx"
description: "Boardsport"
score: 0.7298285